In [1]:
# verify ollama

import psutil

def check_if_running(process_name):
    running = False

    for proc in psutil.process_iter(['name']):
        if process_name in proc.info['name']:
            running = True
            break

    return running

In [2]:
ollama_running = check_if_running('ollama')

if not ollama_running:
    raise RuntimeError("Ollama not running.")

print(f'Ollama running: {check_if_running('ollama')}')

Ollama running: True


In [3]:
import json
from tqdm import tqdm

file_path = 'instruction-data-with-response.json'
with open(file_path, 'r') as f:
    test_data = json.load(f)

def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )
    input_text = (
        f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""
    )

    return instruction_text + input_text

In [4]:
# interacting with Ollama API

import urllib.request

def query_model(prompt, model='gemma3:4b', url='http://localhost:11434/api/chat'):
    data = {
        'model': model,
        'messages': [
            {'role': 'user', 'content': prompt}
        ],
        'options': {
            'seed': 123,
            'temperature': 0,
            'num_ctx': 2048,
        }
    }

    payload = json.dumps(data).encode('utf-8')
    request = urllib.request.Request(url, data=payload, method='POST')
    request.add_header('Content-Type', 'application/json')

    response_data = ""
    with urllib.request.urlopen(request) as response:
        while True:
            line = response.readline().decode('utf-8')
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json['message']['content']
    
    return response_data

In [ ]:
model = 'gemma3:270m'
result = query_model('What do Llamas eat?', model)
print(result)